# Evaluation
We want to evaluate our approach

## Needed
We want to define needed components for this UI

In [1]:
import random
import ipywidgets as widgets
from IPython.display import display, clear_output
import math
import datetime

### Trigger refresh of prediction
each action of typing and sending should yield a new updated prediction for best fitting emojis

Initial definition of emojis used later

In [2]:
#locally defined based on the first analysis of parts of our twitter data: resulting in the 20 most used emojis
#we used them for our first approaches of prediction
top_emojis = list("😳😋😀😌😏😔😒😎😢😅😁😉🙌🙏😘😊😩😍😭😂")
#possible initial set of predictions, only used in naive test cases
predictions = ["🤐","🤑","🤒","🤓","🤔","🤕","🤗","🤘"]

#### Advanced Approach
define the classifier for advanced prediction, used for the sentiment prediction

In [3]:
#navigation into right path and generating classifier
import sys
sys.path.append("..")
sys.path.append("../naive_approach")



import simple_approach.simple_twitter_learning as stl
clf_advanced = stl.pipeline_manager.load_from_pipeline_file("/Users/Carsten/DataSets/NLP_LAB/d2v_final/test_d2v_e2.pipeline")

import Tools.Emoji_Distance as ed

Using TensorFlow backend.


[nltk_data] Downloading package punkt to /Users/Carsten/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/Carsten/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /Users/Carsten/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


#### Generate new Sample for online learning / reinforcement learning

In [4]:
def generate_new_training_sample (msg, emoji):
    sentiment = ed.emoji_to_sentiment_vector(emoji)
    
    #TODO message msg could be filtred
    text = msg
    return text, sentiment

#### Naive Approach
for topic related emoji prediction

In [5]:
#sys.path.append("..")
#print(sys.path)

import naive_approach as clf_naive

In [6]:
tmp_dict = clf_naive.prepareData(stem=True)

#### Merge Predictions
combine the predictions of both approaches

In [7]:
def merged_prediction(msg , split = 0.5 , number = 8, target_emojis = top_emojis):
    
    #calc ratio of prediction splitted between advanced aprroach and naive approach
    number_advanced = round(split*number)
    number_naive = round((1-split)*number)
    
    #predict emojis with the naive approach
    prediction_naive , prediction_naive_values = clf_naive.predict(sentence = msg, lookup= tmp_dict, n = number_naive, embeddings = "word2Vec", stem = True)

    #filter 0 values
    tmp1 = []
    tmp2 = []
    epsilon = 0.0001

    for i in range(len(prediction_naive)):
        if(abs(prediction_naive_values[i]) > epsilon):
            tmp1.append(prediction_naive[i])
            tmp2.append(prediction_naive[i])

    prediction_naive = tmp1
    prediction_naive_values = tmp2
    
    if(len(prediction_naive) < number_naive):
        #print("only few matches")
        number_advanced = number - len(prediction_naive)
        
    #print(number, number_advanced, number_naive)
    
    #predict the advanced approach
    sentiment = clf_advanced.predict([msg])
    prediction_advanced = ed.sentiment_vector_to_emoji(sentiment,n_results = number_advanced, custom_target_emojis=target_emojis)
        
    #concat both predictions
    prediction = list(prediction_advanced)+list(prediction_naive)
    
    return prediction[:number]

Actions triggered when something is changed

In [8]:
def trigger_new_prediction(all_chat, current_message):
    global predictions
    
    #random prediction for  initial test
    #random.shuffle(predictions)
    
    #first prediction only using advanced approach
    #sent = clf_advanced.predict([current_message])
    #p = ed.sentiment_vector_to_emoji(sent,n_results = 8, custom_target_emojis=top_emojis)
    
    #merged prediction
    if(current_message != ""):
        p = merged_prediction(msg = current_message, target_emojis=top_emojis)

        predictions = p

## Trigger Prediction for CSV Table

In [9]:
# get table
import pandas as pd
df = pd.read_csv("Evaluation Sentences - Sentiment related sentences.csv")#, sep="\t")
df.head()

,Sentence,prediction,label
0,I am so happy,NaN,p
1,i love my life,NaN,p
2,i really like this sunshine,NaN,p
3,while doing sport i feel free,NaN,p
4,i is terrible to learn when the weather is thi...,NaN,n


In [10]:
all_predictions = []

for index, row in df.iterrows():
    sentence = row["Sentence"]
    #print(sentence)

    trigger_new_prediction(all_chat="", current_message = sentence)
    #print(predictions)
    
    #prediction to string
    tmp_prediction = "".join(predictions)
    
    #construct the preediction column
    all_predictions.append(tmp_prediction)
    

In [11]:
df["prediction"] = all_predictions

df.head()



,Sentence,prediction,label
0,I am so happy,😂😅😢😳😁😌😉😎,p
1,i love my life,😅😂😢😳😁🏩💌🤟,p
2,i really like this sunshine,😅😂😢😳😭😁😌😔,p
3,while doing sport i feel free,😂😅😁😌😉😎😳😢,p
4,i is terrible to learn when the weather is thi...,😂😅😁😉😌😎😳🙅,n


In [12]:
df.to_csv("E_S - sentiment - d2v - w2v - no stemming.csv", sep='\t', encoding='utf-8')